In [1]:
from decimal import Decimal
from src.domain.Query import JobQuery


from src.Logger import Logger
from src.domain.Collection import LiveJobCollection

from src.solvers.GA_Solver import Solver

from src.DataFrameAnalyses import DataFrameChecker

logger = Logger()

In [2]:
# Jobs for 1 days (shifts)
timespan = 60*24*1

jobs = JobQuery.get_by_source_name_max_util_and_lt_arrival(
    source_name="Fisher and Thompson 10x10",
    max_bottleneck_utilization = Decimal("0.90"),
    arrival_limit= timespan
)
#jobs = [job for job in all_jobs if job.earliest_start <=timespan]

jobs_collection = LiveJobCollection(jobs)

In [3]:
solver = Solver(jobs_collection, logger)
solver.solve_model(
    objective="sum_tardiness",
    pop_size=200,
    ngen=5000,
    seed=42)

schedule_jobs_collection = solver.get_schedule()
info = solver.get_solver_info()

print("Solver Info:", info)

Solver Info: {'objective_value': 3435}


In [4]:
df_schedule = schedule_jobs_collection.to_operations_dataframe()
df_schedule

,Job,Routing_ID,Operation,Machine,Start,Processing Time,End,Arrival,Ready Time,Due Date
0,01-09000-0002,01-01,0,M00,1440,43,1483,118,1440,3354
1,01-09000-0002,01-01,1,M02,1831,90,1921,118,1440,3354
2,01-09000-0002,01-01,2,M04,1974,75,2049,118,1440,3354
3,01-09000-0002,01-01,3,M09,2182,11,2193,118,1440,3354
4,01-09000-0002,01-01,4,M03,2196,69,2265,118,1440,3354
...,...,...,...,...,...,...,...,...,...,...
165,01-09000-0011,01-02,5,M05,2892,10,2902,951,1440,3940
166,01-09000-0011,01-02,6,M07,3256,12,3268,951,1440,3940
167,01-09000-0011,01-02,7,M06,3268,89,3357,951,1440,3940
168,01-09000-0011,01-02,8,M09,3357,45,3402,951,1440,3940


In [5]:
DataFrameChecker.check_core_schedule_constraints(df_schedule)
DataFrameChecker.is_duration_correct(df_schedule)

+ No machine conflicts found.
+ All job operations are scheduled in non-overlapping, correct sequence.
+ All operations start at or after the earliest allowed time.
+ All durations match the difference between start and end.


True

In [6]:
from src.DataFrameAnalyses import DataFrameMetricsAnalyser

df_jobs_metrics = schedule_jobs_collection.to_jobs_metrics_dataframe()
DataFrameMetricsAnalyser.get_jobs_metrics_aggregated(df_jobs_metrics, column="Tardiness")

Tardiness_Interval,0,0 - 60,60 - 120,>120
count,17,0,0,0
